In [ ]:
# For NumCosmo
import os
import sys
import gi

gi.require_version('NumCosmo', '1.0')
gi.require_version('NumCosmoMath', '1.0')
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

from scipy.stats import chi2

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import numpy as np
import math
# The corner package is needed to view the results of the MCMC analysis
import corner



In [ ]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [ ]:
nwalkers = 200
burnin   = 70
thin     = 1

mcat  = Ncm.MSetCatalog.new_from_file_ro ("../Optimized/tools/test_pantheon_bao_aps_200.fits",   nwalkers * burnin)
mcat2 = Ncm.MSetCatalog.new_from_file_ro ("../Optimized/tools/test_pantheon_bao_stretch_200.fits",   nwalkers * burnin)

In [ ]:
s1d1 = chi2.cdf (1.0, df = 1)
s1d2 = chi2.cdf (4.0, df = 1)
s2d1 = chi2.cdf (1.0, df = 1)
s2d2 = chi2.cdf (4.0, df = 1)
s2d3 = chi2.cdf (9.0, df = 1)
qts = [0.5 - s1d2 / 2.0, 0.5 - s1d1 / 2.0, 0.5, 0.5 + s1d1 / 2.0, 0.5 + s1d2 / 2.0]

In [ ]:
rows = np.array ([mcat.peek_row (i*nwalkers + j).dup_array () for j in range (nwalkers) for i in range (0, int (mcat.len ()/nwalkers), thin)])
params = ["$" + mcat.col_symb (i) + "$" for i in range (mcat.ncols ())]
figure = corner.corner (rows[:,1:], labels = params[1:], reverse = False,
                        quantiles=qts, levels = (s2d1, s2d2, s2d3), 
                        bins = 60, smooth = 0.8, smooth1d = 0.8, color = "red")

rows2 = np.array ([mcat2.peek_row (i*nwalkers + j).dup_array () for j in range (nwalkers) for i in range (0, int (mcat2.len ()/nwalkers), thin)])
params = ["$" + mcat2.col_symb (i) + "$" for i in range (mcat2.ncols ())]
figure = corner.corner (rows2[:,1:], labels = params[1:], reverse = False,
                        levels = (s2d1, s2d2, s2d3), color = "blue",
                        bins = 60, smooth = 0.8, smooth1d = 0.8, fig = figure)

figure.set_size_inches(16, 10)


In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))
#ax.set(xlim=(-10, 14), ylim=(-10, 150))

x_index = 1
y_index = 3

scat = ax.scatter(rows[0:nwalkers,x_index], rows[0:nwalkers,y_index], s = 4, c = "red")
scat2 = ax.scatter(rows2[0:nwalkers,x_index], rows[0:nwalkers,y_index], s = 4, c = "blue")

nframes = 600
b = np.unique (np.floor (np.linspace (0, rows.shape[0]/nwalkers, nframes)).astype(int))
nframes = len (b)

def animate(i):    
    x_i = rows[nwalkers * b[i] : nwalkers * b[i + 1], x_index]
    y_i = rows[nwalkers * b[i] : nwalkers * b[i + 1], y_index]
    x2_i = rows2[nwalkers * b[i] : nwalkers * b[i + 1], x_index]
    y2_i = rows2[nwalkers * b[i] : nwalkers * b[i + 1], y_index]
    scat.set_offsets(np.c_[x_i, y_i])
    scat2.set_offsets(np.c_[x2_i, y2_i])

anim = FuncAnimation (fig, animate, interval=50, frames=nframes-1)

In [ ]:
#HTML (anim.to_html5_video ())

In [ ]:
n = min (int(rows.shape[0]/nwalkers), int(rows2.shape[0]/nwalkers))
sv = Ncm.StatsVec.new (1, Ncm.StatsVecType.VAR, False)

from scipy.signal import savgol_filter
import scipy.fftpack

def running_mean(x, N):
    cumsum = np.cumsum (np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

for col in range (0, mcat.ncols ()):
    
    sv.reset (True)
    tevar = []
    for i in range (n-1):
        for w in range (nwalkers):
            sv.set (0, rows[i*nwalkers + w, col])
            sv.update_weight (1.0)
        tevar.append (sv.get_mean (0))
        sv.reset (True)

    sv.reset (True)
    
    tevar2 = []
    for i in range (n-1):
        for w in range (nwalkers):
            sv.set (0, rows2[i*nwalkers + w, col])
            sv.update_weight (1.0)
        tevar2.append (sv.get_mean (0))
        sv.reset (True)

    nrm = 101
    por = 3
    plt.figure (dpi = 160)
    ftevar  = savgol_filter (tevar,  nrm, por)
    ftevar2 = savgol_filter (tevar2, nrm, por)
    
    plt.plot (ftevar,  c = 'red',  lw = 0.5)
    plt.plot (ftevar2, c = 'blue', lw = 0.5)
    plt.axhline (y = np.mean (tevar), color='r', linestyle='-', lw = 0.3)
    plt.axhline (y = np.mean (tevar2), color='b', linestyle='-', lw = 0.3)
    #plt.grid ()
    plt.show ()
#plt.ylim (3.8, 4.2)